In [50]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import json
import re

url = 'https://www.aonprd.com/'

class ArchetypeScraper:
    def __init__(self, url):
        self.url = url
        self.href = f'ShifterAspects.aspx?ItemName=All'
        self.page = requests.get(url + self.href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")
        self.href2_list = []



    def clean_text(self, text):
        # Replace Unicode right single quotation mark with an apostrophe
        cleaned_text = [line.replace('\u2019', "'").replace('\u2018', "'").replace('\u201c', '"').replace('\u201d', '"') for line in text.split('\n') if line.strip() and line.strip() != "."]
        return ' '.join(cleaned_text).strip()
    
    def remove_parenthesis(self, input_string):
        pattern = r"\(.*?pg\..*?\)"
        result = re.sub(pattern, '', input_string)
        result = re.sub(r'<.*?>', '', result)        

        #this removes parenthesis with only text in them 
        result = re.sub(r'\(\d+\)', '', result)

        return result

        

    def link_searcher(self, a_tags):
        for a in a_tags:
            # Check if the link contains "investigator-talents" in the href attribute
            if "ShifterAspects.aspx?ItemName=" in str(a.get('href', '')):
                # Extract everything within double quotes in the href attribute
                regex = r'href="(.*?)"'
                href_unclean = str(a)
                match = re.search(regex, href_unclean)

                if match:
                    self.href2 = match.group(1)
                    self.href2_list.append(self.href2)
                else:
                    continue
            else:
                continue    


        return self.href2_list

         

# What we want to do is to get the data once we see a <b> tag, use the <b> tag as the key section
# Since it's going to be the same each time we can manually set them (like in the items json file)
    def div_search(self, td_tags):
        
        result_list = []

        for td in td_tags:
            a_tags = td.find_all('a')
            data_dict = {}            
            href2_list = self.link_searcher(a_tags)

        
    def talent_info_search(self):
        # Initialize an empty list to store dictionaries for each iteration
        all_data_dicts = []

        for i, href2 in enumerate(self.href2_list):
            self.detail_page = requests.get(url + href2)
            self.detail_soup = BeautifulSoup(self.detail_page.content, 'html.parser')

            b_tags = self.detail_soup.find_all('b')

            h1_tags = self.detail_soup.find_all('h1')

            for h1 in h1_tags:
                h1_tag = self.remove_parenthesis(h1.string)
                if 'Archive' in h1_tag or 'Aspects' in h1_tag:
                    continue
                else:
                    print(h1_tag)

            # <class 'bs4.element.ResultSet'>

            # Initialize an empty dictionary to store the data for this iteration
            data_dict = {}

            for b_tag in b_tags:
                # Find the next_sibling until we hit the </br> tag
                next_sibling = b_tag.next_sibling
                text_under_b = ""

                while next_sibling and (not isinstance(next_sibling, Tag) or next_sibling.name not in ['br']):
                    if isinstance(next_sibling, Tag):
                        if next_sibling.name == 'a':
                            text_under_b += self.clean_text(next_sibling.get_text(strip=True))
                        elif next_sibling.name == 'br':
                            break
                    else:
                        text_under_b += self.clean_text(next_sibling.strip())
                    
                    next_sibling = next_sibling.next_sibling

                if text_under_b:
                    key = self.clean_text(b_tag.get_text(strip=True))
                    key = self.remove_parenthesis(key)
                    value = self.clean_text(text_under_b.strip())
                    value = self.remove_parenthesis(value)
                    data_dict[key] = value



            # Append the data dictionary to the list for this iteration
            all_data_dicts.append({h1_tag: data_dict})

        # Convert the list of dictionaries to JSON format and print it
        json_output = json.dumps(all_data_dicts, indent=2)
        print(json_output)


    def main(self):        
        td_tags = self.soup.find_all('td')
        self.div_search(td_tags)
        self.talent_info_search()


# Instantiate the ArchetypeScraper class
scraper_instance = ArchetypeScraper(url)

# Call the get_class_info method
scraper_instance.main()

 Bat
 Bear
 Boar
 Bull
 Crocodile
 Deinonychus
 Dolphin
 Dragonfly
 Electric Eel
 Elephant
 Falcon
 Frog
 Giant Wasp
 Horse
 Lion
 Lizard
 Mantis
 Monkey
 Mouse
 Octopus
 Owl
 Peafowl
 Scorpion
 Snake
 Snapping Turtle
 Spider
 Stag
 Tiger
 Wolf
 Wolverine
[
  {
    " Bat": {
      "Description Source": ":Wilderness Origins",
      "Source": "Ultimate Wilderness pg. 28",
      "Minor Form": ": You gain darkvision with a range of 60 feet. If you already have darkvision with this range or greater, the range of your darkvision increases by 30 feet. At 8th level, you gain darkvision with a range of 90 feet instead. If you already have darkvision with this range or greater, your darkvision increases by 30 feet. At 15th level, you gain the 8th-level benefit and gain blindsense with a range of 15 feet or, if you already have blindsense with a range of 15 feet or more, you increase its range by 10 feet.",
      "Major Form": "(): Your shape changes to that of a dire bat . While in this form, yo